Libraries

In [3]:
import numpy as np
# from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tqdm as tqdm

Solving Classical LK 2D system

In [4]:
def LV(x,a,b,c,d):    
    xdot = np.array([a*x[0] - b*x[0]*x[1], d*x[0]*x[1] - c*x[1]])
    return xdot

In [5]:
def RK4(f, x0, t0, tf, dt):
    
    t = np.arange(t0, tf, dt)
    nt = t.size
    
    nx = x0.size
    x = np.zeros((nx,nt))
    
    x[:,0] = x0
    
    for k in range(nt-1):
        k1 = dt*f(t[k], x[:,k])
        k2 = dt*f(t[k] + dt/2, x[:,k] + k1/2)
        k3 = dt*f(t[k] + dt/2, x[:,k] + k2/2)
        k4 = dt*f(t[k] + dt, x[:,k] + k3)
        
        dx=(k1 + 2*k2 + 2*k3 +k4)/6
        x[:,k+1] = x[:,k] + dx;  
    
    return x, t

In [6]:
# Defining the problem
a = 1
b = 1
c = 1
d = 1

f= lambda t,x : LV(x,a,b,c,d)         # lambda is an anonymous function which can take may inputs but returns one output. Same case is with MATLAB denoted by @.
x0 = np.array([0.5,1])                 # initial condition    

# Solving the problem
t0 = 0                                # time unit is second
tf = 15
num_points = 1000
dt = (tf - t0) / (num_points - 1)
x, t = RK4(f, x0, t0, tf, dt)

In [ ]:
# Plot the solution
plt.plot(t, x[0], label='Prey')
plt.plot(t, x[1], label='Predator')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Lotka-Volterra Predator-Prey Model')
plt.legend()
plt.grid(True)
plt.show()

Generating Observational Data 

In [7]:
random_indices = np.random.choice(len(t), size=50, replace=False)

x_obs = [x[0][ind] for ind in random_indices]
y_obs = [x[1][ind] for ind in random_indices]

# Generate Gaussian noise for x_obs and y_obs
noise_x = 0.04 * np.random.randn(len(x_obs))
noise_y = 0.04 * np.random.randn(len(y_obs))

# Add the noise to the observed values
x_obs_noise = [x_obs[ind] + noise_x[ind] for ind in range(len(x_obs))]
y_obs_noise = [y_obs[ind] + noise_y[ind] for ind in range(len(y_obs))]

In [ ]:
plt.figure()
plt.title("Noisy observational data")
plt.scatter([t[ind] for ind in random_indices], x_obs_noise)
plt.scatter([t[ind] for ind in random_indices], y_obs_noise)
plt.plot(t, x[0], label='Exact Prey', color="tab:grey")
plt.plot(t, x[1], label='Exact Predator', color="tab:grey")
plt.xlabel('Time')
plt.ylabel('Population')
plt.grid(True)
plt.show()

Tensors

In [8]:
t_obs = torch.tensor([t[ind] for ind in random_indices], dtype=torch.float32).view(-1,1)
u_obs_x = torch.tensor(x_obs,  dtype=torch.float32).view(-1,1) # Noise?
u_obs_y = torch.tensor(y_obs,  dtype=torch.float32).view(-1,1)

In [9]:
# Create 30 approx. equally spaced points for training 
t_physics = []
for i in range(1000):
    if i % 34 == 0:
        t_physics.append(t[i])
t_physics = torch.tensor(t_physics, dtype=torch.float32).view(-1,1).requires_grad_(True)

In [10]:
# Create 300 approx. equally spaced points for testing 
t_test = []
u_test_x = []
u_test_y = []
for i in range(999):
    if i % 3 == 0:
        u_test_x.append(x[0][i])
        u_test_y.append(x[1][i])
        t_test.append(t[i])
t_test = torch.tensor(t_test, dtype=torch.float32).view(-1,1).requires_grad_(True)
u_test_x = torch.tensor(u_test_x, dtype=torch.float32).view(-1,1)
u_test_y = torch.tensor(u_test_y, dtype=torch.float32).view(-1,1)

Defining the Network

In [11]:
class FCN(nn.Module):
    "Defines a standard fully-connected network in PyTorch with sinusoidal activation"
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce_predator = nn.Linear(N_HIDDEN, N_OUTPUT)
        self.fce_prey = nn.Linear(N_HIDDEN, N_OUTPUT)

    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        prey_output = self.fce_prey(x)
        predator_output = self.fce_predator(x)
        return prey_output, predator_output

Training the PINN to learn a parameter

In [ ]:
# define a neural network to train
pinn = FCN(1,1,32,3)

# LEARNABLE PARAMS
a = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
a_s = []
b = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
b_s = []
c = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
c_s = []
d = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
d_s = []

# Add a to the optimiser
optimiser = torch.optim.Adam(list(pinn.parameters())+[a,b,c,d],lr=1e-3)

# Initialize lists to store loss values
physics_loss_values = []
data_loss_values = []
total_loss_values = []

for i in tqdm.tqdm(range(12501)):
    optimiser.zero_grad()

    # Hyperparameter
    lambda1 = 1e5

    # -----------------------
    #       PHYSICS LOSS
    # -----------------------
    u1,u2 = pinn(t_physics)

    # Compute the derivatives with respect to time
    du1dt = torch.autograd.grad(u1, t_physics, torch.ones_like(u1), create_graph=True)[0]
    du2dt = torch.autograd.grad(u2, t_physics, torch.ones_like(u2), create_graph=True)[0]

    # Compute the physics loss for Lotka-Volterra equations
    phy_loss_x = torch.mean((du1dt - a*u1 + b*u1*u2) ** 2)
    phy_loss_y = torch.mean((du2dt + c*u2 - d*u1*u2) ** 2)
    total_physics_loss = phy_loss_x + phy_loss_y

    # -----------------------
    #       DATA LOSS
    # -----------------------
    # Compute the PINN output
    u1, u2 = pinn(t_obs)
    # Compute the data loss for the first equation (x)
    data_loss_x = torch.mean((u1 - u_obs_x)**2)
    # Compute the data loss for the second equation (y)
    data_loss_y = torch.mean((u2 - u_obs_y)**2)

    total_data_loss = data_loss_x + data_loss_y

    # Backpropagate joint loss, take optimiser step
    loss = total_physics_loss + lambda1*total_data_loss
    loss.backward()
    optimiser.step()

    # Append loss values to lists
    physics_loss_values.append(total_physics_loss.item())
    data_loss_values.append(total_data_loss.item())
    total_loss_values.append(loss.item())

    # Keep track of parameter values
    a_s.append(a.item())
    b_s.append(b.item())
    c_s.append(c.item())
    d_s.append(d.item())

    # plot the result as training progresses
    if i % 5000 == 0: 
        u1, u2 = pinn(t_test)
        with torch.no_grad():
            plt.scatter(t_obs, u_obs_x, label="Noisy observations X", alpha=0.3)
            plt.scatter(t_obs, u_obs_y, label="Noisy observations Y", alpha=0.3)
            plt.plot(t_test, u1, label="PINN solution X", color="tab:green")
            plt.plot(t_test, u2, label="PINN solution Y", color="tab:blue")  
            plt.title(f"Training step {i}")
            plt.legend()
            plt.show()

In [ ]:
# Compute model predictions
with torch.no_grad():
    u1_pred, u2_pred = pinn(t_test)

# Compute MSE between model predictions and exact solution
mse_u1 = torch.mean((u1_pred - u_test_x)**2).item()
mse_u2 = torch.mean((u2_pred - u_test_y)**2).item()

print(f"MSE for u1: {mse_u1}")
print(f"MSE for u2: {mse_u2}")

In [ ]:
# Plot loss values over time
plt.figure(figsize=(10, 5))
plt.plot(range(12501), physics_loss_values, label='Physics Loss')
plt.plot(range(12501), data_loss_values, label='Data Loss')
plt.plot(range(12501), total_loss_values, label='Total Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss Function Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure()
plt.title("Parameter a")
plt.plot(a_s, label="PINN estimate")
plt.hlines(1, 0, len(a_s), label="True value", color="tab:green")
plt.legend()
plt.xlabel("Training step")
plt.show()

In [ ]:
plt.figure()
plt.title("Parameter b")
plt.plot(b_s, label="PINN estimate")
plt.hlines(1, 0, len(b_s), label="True value", color="tab:green")
plt.legend()
plt.xlabel("Training step")
plt.show()

In [ ]:
plt.figure()
plt.title("Parameter c")
plt.plot(c_s, label="PINN estimate")
plt.hlines(1, 0, len(c_s), label="True value", color="tab:green")
plt.legend()
plt.xlabel("Training step")
plt.show()

In [ ]:
plt.figure()
plt.title("Parameter d")
plt.plot(d_s, label="PINN estimate")
plt.hlines(1, 0, len(d_s), label="True value", color="tab:green")
plt.legend()
plt.xlabel("Training step")
plt.show()

In [ ]:
print(a,b,c,d)